# Lesson 6b: Factorization Machines with Keras

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Okay bitte :((((

In [2]:
class TopPopRecommender():

    def fit(self, train):

        item_popularity = train[['movie_id','rating']].groupby(by='movie_id').count()

        self.train = train
        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popular_items = item_popularity.sort_values(by='rating',ascending=False).index


    def predict_top(self, user_id, at=5, remove_seen=True):

        if remove_seen:
            seen_items = self.train[self.train.user_id==user_id].movie_id.values
            unseen_items_mask = np.in1d(self.popular_items, seen_items, assume_unique=True, invert = True)
            unseen_items = self.popular_items[unseen_items_mask]
            recommended_items = unseen_items[0:at]

        else:
            recommended_items = self.popular_items[0:at]

        return recommended_items

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pathlib import Path
from zipfile import ZipFile
!mkdir models
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

In [4]:
from google.colab import drive
drive.mount('/content/drive')
complete_train = pd.read_csv('/content/drive/My Drive/_Universität_HPI/Semester 6/Recommenders/Assignments/Assignment 1/data/train.csv')
complete_train_og = complete_train.copy()
test = pd.read_csv('/content/drive/My Drive/_Universität_HPI/Semester 6/Recommenders/Assignments/Assignment 1/givenExample/kaggle_baseline.csv')

Mounted at /content/drive


In [5]:
def text2seq(text, n_genre):
    """ using tokenizer to encoded the multi-level categorical feature
    """
    tokenizer = Tokenizer(lower=True, split='|',filters='', num_words=n_genre)
    tokenizer.fit_on_texts(text)
    seq = tokenizer.texts_to_sequences(text)
    seq = pad_sequences(seq, maxlen=3,padding='post')
    return seq

In [6]:
df = complete_train.copy()
df = df.rename(columns={"release_date": "movie_genre", "age": "sex", "sex": "age"})
df

,user_id,title,movie_id,rating,movie_genre,age,sex
0,2592,Top Gun (1986),1101,4,Action|Romance,50,M
1,4318,12 Angry Men (1957),1203,4,Drama,25,M
2,2756,Robocop 2 (1990),2986,2,Action|Crime|Sci-Fi,18,M
3,1706,Modern Times (1936),3462,5,Comedy,25,M
4,4813,Milk Money (1994),276,3,Comedy|Romance,35,F
...,...,...,...,...,...,...,...
800162,59,"Big Chill, The (1983)",2352,4,Comedy|Drama,50,F
800163,4458,So I Married an Axe Murderer (1993),543,4,Comedy|Romance|Thriller,25,F
800164,1234,Almost Famous (2000),3897,4,Comedy|Drama,18,M
800165,4864,"Fish Called Wanda, A (1988)",1079,5,Comedy,18,M


In [7]:
n_genre = 18
df['movie_genre'] = text2seq(df['movie_genre'].values, n_genre=n_genre).tolist()

In [8]:
df

,user_id,title,movie_id,rating,movie_genre,age,sex
0,2592,Top Gun (1986),1101,4,"[3, 6, 0]",50,M
1,4318,12 Angry Men (1957),1203,4,"[2, 0, 0]",25,M
2,2756,Robocop 2 (1990),2986,2,"[3, 8, 5]",18,M
3,1706,Modern Times (1936),3462,5,"[1, 0, 0]",25,M
4,4813,Milk Money (1994),276,3,"[1, 6, 0]",35,F
...,...,...,...,...,...,...,...
800162,59,"Big Chill, The (1983)",2352,4,"[1, 2, 0]",50,F
800163,4458,So I Married an Axe Murderer (1993),543,4,"[1, 6, 4]",25,F
800164,1234,Almost Famous (2000),3897,4,"[1, 2, 0]",18,M
800165,4864,"Fish Called Wanda, A (1988)",1079,5,"[1, 0, 0]",18,M


In [9]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size=0.2, random_state=7)

In [10]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

def define_input_layers():
    # numerical features
    fea3_input = Input((1,), name = 'input_fea3')
    num_inputs = [fea3_input]
    # single level categorical features
    uid_input = Input((1,), name = 'input_uid') #user_id
    mid_input = Input((1,), name= 'input_mid')  #movie_id
    cat_sl_inputs = [uid_input, mid_input]

    # multi level categorical features (with 3 genres at most)
    genre_input = Input((3,), name = 'input_genre')
    cat_ml_inputs = [genre_input]

    inputs = num_inputs + cat_sl_inputs + cat_ml_inputs

    return inputs

inputs = define_input_layers()

In [11]:
def Tensor_Mean_Pooling(name = 'mean_pooling', keepdims = False):
    return Lambda(lambda x: K.mean(x, axis = 1, keepdims=keepdims), name = name)

def fm_1d(inputs, n_uid, n_mid, n_genre):

    # user feat3 + user embedding + movie embedding + genre embedding
    fea3_input, uid_input, mid_input, genre_input = inputs

    # all tensors are reshape to (None, 1)
    num_dense_1d = [Dense(1, name = 'num_dense_1d_fea4')(fea3_input)]
    cat_sl_embed_1d = [Embedding(n_uid + 1, 1, name = 'cat_embed_1d_uid')(uid_input),
                        Embedding(n_mid + 1, 1, name = 'cat_embed_1d_mid')(mid_input)]
    cat_ml_embed_1d = [Embedding(n_genre + 1, 1, mask_zero=True, name = 'cat_embed_1d_genre')(genre_input)]

    cat_sl_embed_1d = [Reshape((1,))(i) for i in cat_sl_embed_1d]
    cat_ml_embed_1d = [Tensor_Mean_Pooling(name = 'embed_1d_mean')(i) for i in cat_ml_embed_1d]

    # add all tensors
    y_fm_1d = Add(name = 'fm_1d_output')(num_dense_1d + cat_sl_embed_1d + cat_ml_embed_1d)

    return y_fm_1d

y_1d = fm_1d(inputs, 10, 10, 10)

In [12]:
def fm_2d(inputs, n_uid, n_mid, n_genre, k):

    fea3_input, uid_input, mid_input, genre_input = inputs

    num_dense_2d = [Dense(k, name = 'num_dense_2d_fea3')(fea3_input)] # shape (None, k)
    num_dense_2d = [Reshape((1,k))(i) for i in num_dense_2d] # shape (None, 1, k)

    cat_sl_embed_2d = [Embedding(n_uid + 1, k, name = 'cat_embed_2d_uid')(uid_input),
                       Embedding(n_mid + 1, k, name = 'cat_embed_2d_mid')(mid_input)] # shape (None, 1, k)

    cat_ml_embed_2d = [Embedding(n_genre + 1, k, name = 'cat_embed_2d_genre')(genre_input)] # shape (None, 3, k)
    cat_ml_embed_2d = [Tensor_Mean_Pooling(name = 'cat_embed_2d_genure_mean', keepdims=True)(i) for i in cat_ml_embed_2d] # shape (None, 1, k)

    # concatenate all 2d embed layers => (None, ?, k)
    embed_2d = Concatenate(axis=1, name = 'concat_embed_2d')(num_dense_2d + cat_sl_embed_2d + cat_ml_embed_2d)

    # calcuate the interactions by simplication
    # sum of (x1*x2) = sum of (0.5*[(xi)^2 - (xi^2)])
    tensor_sum = Lambda(lambda x: K.sum(x, axis = 1), name = 'sum_of_tensors')
    tensor_square = Lambda(lambda x: K.square(x), name = 'square_of_tensors')

    sum_of_embed = tensor_sum(embed_2d)
    square_of_embed = tensor_square(embed_2d)

    square_of_sum = Multiply()([sum_of_embed, sum_of_embed])
    sum_of_square = tensor_sum(square_of_embed)

    sub = Subtract()([square_of_sum, sum_of_square])
    sub = Lambda(lambda x: x*0.5)(sub)
    y_fm_2d = Reshape((1,), name = 'fm_2d_output')(tensor_sum(sub))

    return y_fm_2d, embed_2d

y_fm2_d, embed_2d = fm_2d(inputs, 10, 10, 10, 5)

In [13]:
def fm_model(n_uid, n_mid, n_genre, k, dnn_dr):

    inputs = define_input_layers()

    y_fm_1d = fm_1d(inputs, n_uid, n_mid, n_genre)
    y_fm_2d, embed_2d = fm_2d(inputs, n_uid, n_mid, n_genre, k)


    # combinded deep and fm parts
    y = Concatenate()([y_fm_1d, y_fm_2d])
    y = Dense(1, name = 'fm_output')(y)

    fm_model_1d = Model(inputs, y_fm_1d)
    fm_model_2d = Model(inputs, y_fm_2d)
    fm_model = Model(inputs, y)

    return fm_model_1d, fm_model_2d, fm_model

In [14]:
params = {
    'n_uid': df['user_id'].max(),
    'n_mid': df['movie_id'].max(),
    'n_genre': 18,
    'k':20,
    'dnn_dr': 0.5
}

fm_model_1d, fm_model_2d, fm_model = fm_model(**params)

In [15]:
def df2xy(ratings):
    x = [df['age'].values,
         df['user_id'].values,
         df['movie_id'].values,
         np.concatenate(df['movie_genre'].values).reshape(-1,3)]
    y = df['rating'].values
    return x,y

train_x, train_y = df2xy(train)
valid_x, valid_y = df2xy(val)

In [ ]:
from tensorflow.keras.callbacks import  EarlyStopping, ModelCheckpoint
fm_model_2d.compile(
    loss=tf.keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(learning_rate=0.001)
)
early_stop = EarlyStopping(monitor='val_loss', patience=3)
model_ckp = ModelCheckpoint(filepath='./models/deepfm_weights.h5',
                            monitor='val_loss',
                            save_weights_only=True,
                            save_best_only=True)
callbacks = [model_ckp,early_stop]
train_history = fm_model_2d.fit(train_x, train_y,
                                  epochs=100, batch_size=2048,
                                  validation_data=(valid_x, valid_y),
                                  callbacks = callbacks)



 12/391 [..............................] - ETA: 1:27 - loss: 7.0665

In [ ]:
def evaluate_model(model, test_x, test_y):
    # Make predictions on the test data
    predictions = model.predict(test_x)
    print("got here")
    predictions_flattened = predictions.flatten()
    # Calculate MSE
    #mse = np.mean((predictions_flattened - test_y) ** 2)
    mse = tf.keras.losses.mean_squared_error(test_y, predictions_flattened)
    mse_value = np.mean(mse.numpy())
    return mse_value

In [ ]:
what = evaluate_model(fm_model_2d, valid_x, valid_y)
what

25006/25006 [==============================] - 49s 2ms/step
got here


1.047777

0.4588

In [ ]:
topPopular = TopPopRecommender()
topPopular.fit(complete_train_og)
user_ages_lookup = df[['user_id', 'age']].drop_duplicates().set_index('user_id')['age'].to_dict()
unique_list_agg = lambda x: list(set(x.apply(tuple)))
movie_genre_lookup = df.groupby('movie_id')['movie_genre'].agg(unique_list_agg).to_dict()



def get_recommendations(model, user_id, at=25):
    unseen_movie_ids_array = np.array(topPopular.predict_top(user_id, at=200, remove_seen=True))
    user_age = user_ages_lookup.get(user_id, 0)
    age_array = np.full(len(unseen_movie_ids_array), user_age)
    id_array = np.full(len(unseen_movie_ids_array), user_id)
    movie_genre_array = np.array([list(movie_genre_lookup.get(id)[0]) for id in unseen_movie_ids_array])
    input = [age_array, id_array, unseen_movie_ids_array, movie_genre_array]

    # Predict with the model
    prediction = model.predict(input)
    pred_flattened = prediction.flatten()
    combined = np.column_stack((unseen_movie_ids_array, pred_flattened))
    sorted_combined = combined[np.argsort(combined[:, 1])[::-1]]
    sorted_ids = sorted_combined[:, 0].astype(int)
    sorted_ratings = sorted_combined[:, 1].reshape(-1, 1)

    return sorted_ids[:at]

In [ ]:
get_recommendations(fm_model_2d, 2, 25)

7/7 [==============================] - 0s 2ms/step


array([3671,   50, 2762,  858, 1250,  904, 1221,  923, 1136,  919, 1242,
        912,  908,  913, 1276, 2804, 1358, 1234,   47,  296, 1225,  260,
       2571, 1294,  527])